In [9]:
#!pip install selenium
#!pip install webdriver_manager
from selenium import webdriver

from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import codecs
from selenium.webdriver.common.keys import Keys
import re

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select

In [10]:
website = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
path = 'C:/Users/olive/Downloads/chromedriver'
driver = webdriver.Chrome(path)
driver.get(website)

C:\Users\olive\AppData\Local\Temp\ipykernel_24020\2955384389.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [12]:
driver.find_elements_by_tag_name('h3')

UnexpectedTagNameException: Message: Select only works on <select> elements, not on a


In [ ]:
driver.quit()